In [1159]:
## Reference: https://www.youtube.com/watch?v=WgPbbWmnXJ8&ab_channel=Murtaza%27sWorkshop-RoboticsandAI

# 1. Preparing Working Directory

## 1.1 Initialization of Data

** need to run on every execution / after every restart of the kernel

In [1160]:
import os

In [1161]:
# Working Paths
base_path = '.'
paths = {
    'WORKSPACE_PATH': os.path.join(base_path, 'workspace'),
    
        'IMAGE_PATH': os.path.join(base_path, 'workspace', 'images'),
        'VIDEO_PATH': os.path.join(base_path, 'workspace', 'videos'),
        'MODEL_PATH': os.path.join(base_path, 'workspace', 'models'),
    
    'LIB': os.path.join(base_path, 'lib')
}

In [1162]:
# Files
files = {
    'REQUIREMENTS': os.path.join(base_path, 'workspace', 'requirements.txt')
}

In [1163]:
def build_path(path: str) -> bool:
    if not os.path.exists(path):
        if os.name == 'posix': # Unix-based systems
            !mkdir -p {path}
        if os.name == 'nt':    # Windows systems
            !mkdir {path}
        return True
    return False

                
for path in paths.values():
    build_path(path)

# 2. Installation of Dependencies

Please perform the following in command line:

`pip install -r workspace/requirements.txt`

# 3. Model Application

## 3.1 Setting Up

** need to run on every execution

In [1164]:
from ultralytics import YOLO
import cv2

In [1165]:
model = YOLO(os.path.join(paths['MODEL_PATH'], 'yolov8n.pt'))

## Display Styles

In [1166]:
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

In [1167]:
import random
import colorsys
import math

def to_rgb(h, s, v):
    return [int(256*i) for i in colorsys.hsv_to_rgb(h,s,v)]

def gen_color():
    h,v,s = random.random(), 0.75+random.random()/4.0, .8
    r,g,b = to_rgb(h,s,v)
    return (b,g,r)

In [1168]:
BOX_COLOR = {_cls: gen_color() for _cls in classNames}

## 3.2 Applying on Static Image

** Prerequisite: Sections 1.1, 3.1

### Parameters / Image File Path

In [1169]:
file_path = '3.png'
img_path = os.path.join(paths['IMAGE_PATH'], file_path)

# or directly do img_path = (some path)

### Execution

In [1170]:
results = model(img_path, show=False)

img = cv2.imread(img_path)
img = cv2.convertScaleAbs(img, alpha=0.5, beta=0)

for r in results:
    boxes = r.boxes
    for box in boxes:
        _cls = classNames[int(box.cls[0])]
        conf = float(box.conf[0])
        
        if conf > 0.3:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            disp_text = f'{_cls} {math.ceil((conf*100))/100}'

            # adding rectangles to the image
            cv2.rectangle(img, (x1,y1), (x2,y2), BOX_COLOR[_cls], 2)

            cv2.putText(img, disp_text, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, BOX_COLOR[_cls], 2)
cv2.imshow('image', img)
cv2.waitKey(0)


image 1/1 d:\kht3327\_Projects\Previous Work\ObjectDetection-v1\workspace\images\3.png: 384x640 15 cars, 8 motorcycles, 53.3ms
Speed: 2.5ms preprocess, 53.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


113

## 3.3 Applying on Live Video

** Prerequisite: Sections 1.1, 3.1

In [1171]:
import math
import cvzone

In [1172]:
if os.name=='posix':
    !ls {os.path.join(paths['VIDEO_PATH'])}
if os.name=='nt':
    !dir /b {os.path.join(paths['VIDEO_PATH'])}

bikes.mp4
cars.mp4
mask
motorbikes.mp4
people.mp4
ppe-1.mp4
ppe-2.mp4
ppe-3.mp4


## 3.4 Task 1: Object Detection

** Prerequisite: Sections 1.1, 3.1, 3.3

### Parameters / Video File Path

In [1173]:
vid_name = 'cars.mp4'
vid_path = os.path.join(paths['VIDEO_PATH'], vid_name)

# or directly do vid_path = (some path)

### Execution

In [1174]:
cap = cv2.VideoCapture(vid_path)
cap.set(3, 640)
cap.set(4, 480)
cv2.namedWindow('image', cv2.WINDOW_NORMAL)

hei = DEF_HEIGHT = 600
pause = False

while True:
    key = cv2.waitKey(1)
    if key & 0xFF == ord(' '):
        pause = not pause
    if key & 0xFF == ord('q'):
        break    
    
    if pause:    
        continue    
        
    success, img = cap.read()
    
    if not success:
        break
    
    h, w, _ = img.shape
    aspect_ratio = w / h
    wid = int(hei * aspect_ratio)
    
    results = model(img, stream=True, show=False)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            cls = classNames[int(box.cls[0])]
            conf = float(box.conf[0])
            
            if conf > 0.3:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                disp_text = f'{cls} {math.ceil((conf*100))/100}'

                # adding rectangles to the image
                cv2.rectangle(img, (x1,y1), (x2,y2), BOX_COLOR[cls], 2)

                cv2.putText(img, disp_text, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, BOX_COLOR[cls], 2)

    # cv2.resizeWindow('image', wid, hei)
    cv2.imshow('image', img)
    
        
cap.release()
cv2.destroyAllWindows()


0: 384x640 10 cars, 1 truck, 49.1ms
Speed: 1.1ms preprocess, 49.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 40.2ms
Speed: 1.5ms preprocess, 40.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 truck, 42.3ms
Speed: 1.1ms preprocess, 42.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 truck, 36.8ms
Speed: 1.5ms preprocess, 36.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 truck, 37.3ms
Speed: 1.0ms preprocess, 37.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 bus, 39.9ms
Speed: 0.5ms preprocess, 39.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 bus, 1 truck, 41.7ms
Speed: 1.6ms preprocess, 41.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


## 3.5 Task 2: Instance Identification

** Prerequisite: Sections 1.1, 3.1, 3.3

In [1175]:
import numpy as np

In [1176]:
def get_id_color(id: int):
    h,v,s = id/100 % 1, 0.75 + (id/1000 % 1) / 4, 0.5
    r,g,b = to_rgb(h,s,v)
    return (b,g,r)

### 3.5.1 Method 1:  Abewley Sort Library

In [1177]:
## References: 
##   - https://github.com/abewley/sort
##   - https://arxiv.org/pdf/1602.00763.pdf

In [1178]:
# Fetching the library from github
paths['LIB_SORT'] = os.path.join(paths['LIB'], 'abewley_sort')
if build_path(paths['LIB_SORT']):
    !git clone https://github.com/abewley/sort {paths['LIB_SORT']}

In [1179]:
# Importing the library
import sys
sys.path.append(paths['LIB_SORT'])
from sort import *

In [1180]:
# Tracking using the Abewley Sort library
## max_age: the number of frames where the item is lost so that we discard the item
## iou: intersection over union, measures how good the bounding box matches with the ground truth
tracker = Sort(max_age=20, min_hits=1, iou_threshold=0.05)

In [1181]:
cap = cv2.VideoCapture(vid_path)
cap.set(3, 640)
cap.set(4, 480)
cv2.namedWindow('image', cv2.WINDOW_NORMAL)

hei = DEF_HEIGHT = 600
pause = False

try: 
    while True:
        key = cv2.waitKey(1)
        if key & 0xFF == ord(' '):
            pause = not pause
        if key & 0xFF == ord('q'):
            break    

        if pause:    
            continue    

        success, img = cap.read()

        if not success:
            break

        h, w, _ = img.shape
        aspect_ratio = w / h
        wid = int(hei * aspect_ratio)

        results = model(img, stream=True, show=False)

        # tracker detections array init
        detections = np.empty((0, 5))

        for r in results:
            boxes = r.boxes
            for box in boxes:
                cls = classNames[int(box.cls[0])]
                conf = float(box.conf[0]) # box.conf[0] is a Tensor float format (NOT a float)

                if conf > 0.2:
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    disp_text = f'{cls} {math.ceil((conf*100))/100}'

                    # adding rectangles to the image
                    # cv2.rectangle(img, (x1,y1), (x2,y2), BOX_COLOR[cls], 2)

                    cv2.putText(img, disp_text, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, BOX_COLOR[cls], 2)

                    # adding the rect into the tracker detection list
                    currentArray = np.array([x1, y1, x2, y2, conf])
                    detections = np.vstack((detections, currentArray))

        # Update the tracker (follows the format np.empty((0,5)): [x1,y1,x2,y2,confidence] -> [x1,y1,x2,y2,id])
        tracker_results = tracker.update(detections)

        for r in tracker_results:
            x1, y1, x2, y2, Id = r
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # adding rectangles to the image
            cv2.rectangle(img, (x1,y1), (x2,y2), get_id_color(Id), 2)

            cv2.putText(img, f'{int(Id)}', (x1, y1-25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, get_id_color(Id), 2)

        cv2.resizeWindow('image', wid, hei)
        cv2.imshow('image', img)

finally:
    cap.release()
    cv2.destroyAllWindows()


0: 384x640 10 cars, 1 truck, 40.0ms
Speed: 1.0ms preprocess, 40.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 39.7ms
Speed: 1.1ms preprocess, 39.7ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 truck, 38.9ms
Speed: 2.1ms preprocess, 38.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 truck, 38.0ms
Speed: 0.5ms preprocess, 38.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 truck, 48.8ms
Speed: 1.0ms preprocess, 48.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


### 3.5.2 Method 2: Custom Instancing Algorithm

In [1182]:
# only works in low-speed environment
# the algorithm works for O(n**2)

class InstanceDetector:
    
    def __init__(self, maxage=5, checkage=2, strict_detection=True):
        self.maxage = maxage
        self.checkage = checkage
        self.strict_detection = strict_detection
        self.instances = dict() # format: {id: [cx, cy, w/2, h/2, disappeared_frame_count], {cls: conf, ...}}
        self.instance_cnt = 1

    def transform(self, detection=np.empty((6,))):
        x1, y1, x2, y2, cls, conf = detection
        cx, cy, hw, hh = (x1+x2)/2, (y1+y2)/2, (x1-x2)/2, (y1-y2)/2
        return np.array([cx, cy, hw, hh, cls, conf, -1])
        
    def update(self, detections=np.empty((0,6))):
        """
        Required input format: [x1, y1, x2, y2, class, conf]
        """
        try:
            transformed = np.apply_along_axis(self.transform, 1, detections)
        except:
            transformed = np.empty((0,7))
        """
        Required transformed format: [cx, cy, hw, hh, class, conf, -1]
        """
        
        instances = dict(self.instances)
        for new_instance in transformed:
            cx, cy, hw, hh, cls, conf, _ = new_instance
            area = hw * hh
            aspect = hw / hh
            
            new_instance = np.array([cx, cy, hw, hh, -1])
            min_sqd = -1
            min_instance = -1
            
            destroyed = False
            
            for Id in instances:
                old_instance, labels = instances[Id]
                ocx, ocy, ohw, ohh, age = old_instance
                oarea = ohw * ohh
                oaspect = ohw / ohh
                dx = cx - ocx
                dy = cy - ocy
                
                area_ratio = area / oarea
                d_aspect = aspect / oaspect
                d_aspect = 1 / d_aspect if d_aspect < 1 else d_aspect
                sqdist = dx ** 2 + dy ** 2 + (d_aspect-1)
                
                if abs(dx) > max(abs(ohw), abs(hw)) * 2 or abs(dy) > max(abs(ohh), abs(hh)) * 2:
                    continue
                elif not 0.01 < area_ratio < 100 or d_aspect > 5 or age > self.checkage:
                    destroyed = True
                    continue
                elif self.strict_detection and age == -1  and (abs(dx) > max(abs(ohw), abs(hw)) * .2 or abs(dy) > max(abs(ohh), abs(hh)) * .2):
                    continue
                    
                min_sqd = sqdist if min_sqd == -1 else min(min_sqd, sqdist)
                if min_sqd == sqdist:
                    min_instance = Id
                    destroyed = False
                
            if min_instance != -1 and min_sqd < new_instance[2]**2+new_instance[3]**2:
                old_instance, labels = self.instances[min_instance]
                labels[cls] = max(labels[cls], conf) if cls in labels else conf
                self.instances[min_instance] = new_instance, labels
            elif not destroyed:
                self.instances[self.instance_cnt] = new_instance, {cls: conf}
                self.instance_cnt += 1
        
        for Id in self.instances:
            self.instances[Id][0][-1] += 1
        self.instances = {Id: instance for Id, instance in self.instances.items() if instance[0][-1] < self.maxage}
                
        """
        Required output format: [cx, cy, hw, hh, cls, conf, disappeared_frame_count]
        """
        
        return self.instances

In [1183]:
cap = cv2.VideoCapture(vid_path)
cap.set(3, 640)
cap.set(4, 480)
cv2.namedWindow('image', cv2.WINDOW_NORMAL)

hei = DEF_HEIGHT = 600
pause = False

# tracker init
detector = InstanceDetector(maxage=5)

try:
    while True:
        key = cv2.waitKey(1)
        if key & 0xFF == ord(' '):
            pause = not pause
        if key & 0xFF == ord('q'):
            break    

        if pause:    
            continue    

        success, img = cap.read()

        if not success:
            break

        h, w, _ = img.shape
        aspect_ratio = w / h
        wid = int(hei * aspect_ratio)

        results = model(img, stream=True, show=False)

        # tracker detections array init
        detections = np.empty((0, 6))

        for r in results:
            boxes = r.boxes
            for box in boxes:
                icls = int(box.cls[0])
                cls = classNames[icls]
                conf = float(box.conf[0])

                if conf > 0.05:
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    disp_text = f'{cls} {math.ceil((conf*100))/100}'

                    # adding rectangles to the image
                    # cv2.rectangle(img, (x1+2,y1+2), (x2-2,y2-2), BOX_COLOR[cls], 2)

                    # cv2.putText(img, disp_text, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, BOX_COLOR[cls], 2)

                    # adding the rect into the tracker detection list
                    currentArray = np.array([x1, y1, x2, y2, icls, conf])
                    detections = np.vstack((detections, currentArray))

         # Update the tracker 
        detector_results = detector.update(detections)

        for Id in detector_results:
            instance, labels = detector_results[Id]
            cx, cy, hw, hh, age = instance

            labels = sorted(labels.items(), key=lambda item:item[1], reverse=True)
            labels = [f'{classNames[int(icls)]} {math.ceil((conf*100))/100}' for icls, conf in labels]

            if age != 0:
                continue

            #cls = classNames[int(icls)]
            x1, y1, x2, y2 = int(cx+hw), int(cy+hh), int(cx-hw), int(cy-hh)
            disp_text = f'{labels}'#f'{cls} {math.ceil((conf*100))/100}'

            # adding rectangles to the image
            cv2.rectangle(img, (x1,y1), (x2,y2), get_id_color(Id), 2)

            cv2.putText(img, f'{int(Id)}', (x1, y1-25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, get_id_color(Id), 2)
            cv2.putText(img, disp_text, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, get_id_color(Id), 2)

        cv2.resizeWindow('image', wid, hei)
        cv2.imshow('image', img)

finally:
    cap.release()
    cv2.destroyAllWindows()


0: 384x640 10 cars, 1 truck, 50.6ms
Speed: 1.5ms preprocess, 50.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 45.0ms
Speed: 1.7ms preprocess, 45.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 truck, 40.6ms
Speed: 2.0ms preprocess, 40.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 truck, 42.2ms
Speed: 2.5ms preprocess, 42.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 truck, 41.7ms
Speed: 1.4ms preprocess, 41.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


## 3.6 Task 3: Car Counter

** Prerequisite: Sections 1.1, 3.1, 3.3, 3.5 (header), 3.5.2

** In this section, every sub-sections requires the execution of every sub-section before it.

In [1184]:
paths['FIG_PATH'] = os.path.join(paths['WORKSPACE_PATH'], 'figs')
paths['FIG_PATH_CARCOUNTER'] = os.path.join(paths['FIG_PATH'], 'car_counter')

build_path(paths['FIG_PATH_CARCOUNTER'])

False

In [1185]:
options = {
    'car': 
        {
             'vidname': 'cars.mp4',
             'limits': 
                 {
                     ((250, 363), (675, 363)): {'color':(50,50,255), 'format':'{}', 'position': (255, 100)}
                 }, 
             'recoglist': ['car', 'truck', 'bus', 'motorbike'],
             'maskpng': 'car_mask.png',
             'labelpng': 'car_counter_label.png',
             'labelpos': (0, 0),
             'playback': 1
        },
    'mall': 
        {
             'vidname': 'people.mp4',
             'limits': 
                 {
                     ((103, 161), (296, 161)): {'color':(50,255,50), 'format':'{}', 'position': (929, 345)}, # up
                     ((470, 297), (673, 297)): {'color':(50,50,255), 'format':'{}', 'position': (1191, 345)} # down
                 }, 
             'recoglist': ['person'],
             'maskpng': 'people_mask.png', 
             'labelpng': 'people_counter_label.png',
             'labelpos': (730, 260),
             'playback': 3
        }
}

In [1186]:
option = 'car'
nomask = False

In [1187]:
_data = options[option]
vidname, limits, recoglist, maskpng = _data['vidname'], _data['limits'], _data['recoglist'], _data['maskpng']
labelpng, labelpos = os.path.join(paths['FIG_PATH_CARCOUNTER'], _data['labelpng']), _data['labelpos']

### 3.6.1 Overlap Detection

In [1188]:
def check_overlap(limit: tuple[tuple[int]], rect: list[4]) -> bool:
    """
    Input format requirement: 
     - limit: [(x1, y1), (x2, y2)]
     - rect:  [x1, y1, x2, y2]
    """
    x1, x2 = limit
    x1, y = x1
    x2, _ = x2
    x_bmin, x_bmax = min(x1, x2), max(x1, x2) # line bound coords
    
    x1, y1, x2, y2 = rect
    xmin, xmax = min(x1, x2), max(x1, x2)
    ymin, ymax = min(y1, y2), max(y1, y2)
    
    return not (ymax < y or ymin > y or xmax < x_bmin or xmin > x_bmax)

### 3.6.2 Masking

In [1189]:
paths['MASK_PATH'] = os.path.join(paths['VIDEO_PATH'], 'mask')

** The mask must be of the same size as the video.

In [1190]:
mask = cv2.imread(os.path.join(paths['MASK_PATH'], maskpng))

In [1191]:
# for masking, do
# masked = cv2.bitwise_and(img, mask)

### 3.6.3 Playback Speed Adjustment

In [1192]:
class Playback:
    def __init__(self, playback_speed=1):
        """
        Note: playback speed must be > 1 in this version.
        """
        self.playback_speed = playback_speed
        self.frame_cnt = 0
        self.actual_cnt = 0
    
    def step(self):
        self.frame_cnt += 1
        if self.frame_cnt > self.actual_cnt * self.playback_speed:
            self.actual_cnt += 1
            return True
        return False

In [1193]:
playback_speed = _data['playback'] if 'playback' in _data else 1

### 3.6.4 Showcase

In [1194]:
cap = cv2.VideoCapture(os.path.join(paths['VIDEO_PATH'], vidname))
cv2.namedWindow('image', cv2.WINDOW_NORMAL)

hei = DEF_HEIGHT = 600
pause = False

# tracker init
detector = InstanceDetector(maxage=5, strict_detection=False)

detected = {limit: set() for limit in limits}

playback = Playback(playback_speed=playback_speed)

try:
    while True:
        key = cv2.waitKey(1)
        if key & 0xFF == ord(' '):
            pause = not pause
        if key & 0xFF == ord('q'):
            break    

        if pause:    
            continue    

        success, img = cap.read()

        if not success:
            break
        if not playback.step():
            continue

        h, w, _ = img.shape
        aspect_ratio = w / h
        wid = int(hei * aspect_ratio)

        masked_img = cv2.bitwise_and(img, mask) if not nomask else img
        results = model(masked_img, stream=True, show=False)
        
        labelimg = cv2.imread(labelpng, cv2.IMREAD_UNCHANGED)
        img = cvzone.overlayPNG(img, labelimg, labelpos)

        # tracker detections array init
        detections = np.empty((0, 6))

        for r in results:
            boxes = r.boxes
            for box in boxes:
                icls = int(box.cls[0])
                cls = classNames[icls]
                conf = float(box.conf[0])

                if conf > 0.05 and cls in recoglist:
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    disp_text = f'{cls} {math.ceil((conf*100))/100}'

                    # adding the rect into the tracker detection list
                    currentArray = np.array([x1, y1, x2, y2, icls, conf])
                    detections = np.vstack((detections, currentArray))

         # Update the tracker 
        detector_results = detector.update(detections)
        
        for limit in limits:
            cv2.line(img, limit[0], limit[1], limits[limit]['color'], 2)

        for Id in detector_results:
            instance, labels = detector_results[Id]
            cx, cy, hw, hh, age = instance

            labels = sorted(labels.items(), key=lambda item:item[1], reverse=True)
            labels = [f'{classNames[int(icls)]} {math.ceil((conf*100))/100}' for icls, conf in labels]

            if age != 0:
                continue

            #cls = classNames[int(icls)]
            x1, y1, x2, y2 = int(cx+hw), int(cy+hh), int(cx-hw), int(cy-hh)
            disp_text = f"{' '.join(labels)}"
            
            for limit in limits:
                if check_overlap(limit, [x1, y1, x2, y2]):
                    detected[limit].add(Id)

            # adding rectangles to the image
            cv2.rectangle(img, (x1,y1), (x2,y2), get_id_color(Id), 2)
            for limit in limits:
                if Id in detected[limit]:                
                    cv2.rectangle(img, (x1+2,y1+2), (x2-2,y2-2), limits[limit]['color'], 2)

            cv2.putText(img, f'{int(Id)}', (x1, y1-25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, get_id_color(Id), 2)
            cv2.putText(img, disp_text, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, get_id_color(Id), 2)

        for limit in limits:
            cv2.putText(
                img, 
                limits[limit]['format'].format(len(detected[limit])), 
                limits[limit]['position'], 
                cv2.FONT_HERSHEY_PLAIN, 5, limits[limit]['color'], 7
            )    
        
        cv2.resizeWindow('image', wid, hei)
        cv2.imshow('image', img)
finally:  
    cap.release()
    cv2.destroyAllWindows()


0: 384x640 4 cars, 39.0ms
Speed: 1.1ms preprocess, 39.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 41.1ms
Speed: 0.5ms preprocess, 41.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 39.9ms
Speed: 1.6ms preprocess, 39.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 2 trucks, 39.9ms
Speed: 1.5ms preprocess, 39.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 38.9ms
Speed: 1.0ms preprocess, 38.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 37.5ms
Speed: 1.0ms preprocess, 37.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 38.2ms
Speed: 1.0ms preprocess, 38.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 37.9ms
Speed: 1.1ms preprocess, 37.9ms inference, 0.5ms postprocess per image at sha